In [3]:
pip install google-analytics-data

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [239]:
import pandas as pd
from google.analytics.data_v1beta import BetaAnalyticsDataClient
from google.analytics.data_v1beta.types import RunReportRequest, DateRange, Dimension, Metric
from google.oauth2 import service_account
from datetime import date, timedelta

In [261]:
# Caminhos
CREDENTIALS_JSON = "C:/Users/TI/GA4_Google_Analytics/credenciais/ecommerce-liebe-164d83fe62fc.json"  
GA4_PROPERTY_ID = "373613821"  # ID propriedade GA4
DATE_RANGE = {"start": "2024-01-01", "end": "2024-05-20"}

In [253]:
def get_authenticated_client(credentials_path):
    credentials = service_account.Credentials.from_service_account_file(credentials_path)
    return BetaAnalyticsDataClient(credentials=credentials)

client = get_authenticated_client(CREDENTIALS_JSON)

<h3>BLOCO 1: AQUISICAO_CONVERSAO

In [263]:
def run_bloco1_aquisicao(client, property_id, date_range):
    request = RunReportRequest(
        property=f"properties/{property_id}",
        dimensions=[
            Dimension(name="date"),
            Dimension(name="sessionCampaignName"),
            Dimension(name="sessionSource"),
            Dimension(name="sessionMedium")
        ],
        metrics=[
            Metric(name="activeUsers"),
            Metric(name="advertiserAdClicks"),
            Metric(name="advertiserAdImpressions"),
            Metric(name="advertiserAdCost"),
            Metric(name="purchaseRevenue"),
            Metric(name="transactions")
        ],
        date_ranges=[DateRange(start_date=date_range["start"], end_date=date_range["end"])]
    )

    rows = client.run_report(request).rows
    dados = []
    for row in rows:
        dims = row.dimension_values
        mets = row.metric_values
        dados.append({
            "data": dims[0].value if len(dims) > 0 else "N/A",
            "campanha": dims[1].value if len(dims) > 1 else "N/A",
            "fonte": dims[2].value if len(dims) > 2 else "N/A",
            "meio": dims[3].value if len(dims) > 3 else "N/A",
            "usuarios": mets[0].value if len(mets) > 0 else "0",
            "cliques": mets[1].value if len(mets) > 1 else "0",
            "impressoes": mets[2].value if len(mets) > 2 else "0",
            "custo": mets[3].value if len(mets) > 3 else "0",
            "receita": mets[4].value if len(mets) > 4 else "0",
            "transacoes": mets[5].value if len(mets) > 5 else "0"
        })

    return pd.DataFrame(dados)

In [265]:
df_bloco1 = run_bloco1_aquisicao(client, GA4_PROPERTY_ID, DATE_RANGE)
# Converter as colunas para float
colunas_numericas = ["usuarios", "cliques", "impressoes", "custo", "receita", "transacoes"]
df_bloco1[colunas_numericas] = df_bloco1[colunas_numericas].replace(",", ".", regex=True).apply(pd.to_numeric, errors='coerce')

In [359]:
df_bloco1.to_csv("bloco1_aquisicao_conversao.csv", index=False, encoding="utf-8", sep=";", decimal=",")
df_bloco1.head()

,data,campanha,fonte,meio,usuarios,cliques,impressoes,custo,receita,transacoes
0,20240429,[OPT] Geração Demanda,google,cpc,1422,3658,94101,380.720597,0.000000,0
1,20240223,(referral),l.instagram.com,referral,1345,0,0,0.000000,180.410001,2
2,20240426,e-Plus_google_pmax_geral_produto_conversion_di...,google,cpc,1191,1781,65135,849.367182,644.499996,4
3,20240410,(referral),l.instagram.com,referral,865,0,0,0.000000,511.279999,3
4,20240224,(referral),l.instagram.com,referral,787,0,0,0.000000,156.539998,1


In [269]:
df_bloco1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5070 entries, 0 to 5069
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   data        5070 non-null   object 
 1   campanha    5070 non-null   object 
 2   fonte       5070 non-null   object 
 3   meio        5070 non-null   object 
 4   usuarios    5070 non-null   int64  
 5   cliques     5070 non-null   int64  
 6   impressoes  5070 non-null   int64  
 7   custo       5070 non-null   float64
 8   receita     5070 non-null   float64
 9   transacoes  5070 non-null   int64  
dtypes: float64(2), int64(4), object(4)
memory usage: 396.2+ KB


<h3>BLOCO 2: ENGAJAMENTO

In [345]:
def run_bloco2_engajamento(client, property_id, date_range):
    request = RunReportRequest(
        property=f"properties/{property_id}",
        dimensions=[
            Dimension(name="date"),
            Dimension(name="sessionCampaignName"),
            Dimension(name="sessionSource"),
            Dimension(name="sessionMedium")
        ],
        metrics=[
            Metric(name="sessions"),
            Metric(name="newUsers"),
            Metric(name="totalUsers"),
            Metric(name="engagedSessions"),
            Metric(name="bounceRate"),
            Metric(name="averageSessionDuration")
        ],
        date_ranges=[DateRange(start_date=date_range["start"], end_date=date_range["end"])]
    )

    rows = client.run_report(request).rows
    dados = []
    for row in rows:
        dims = row.dimension_values
        mets = row.metric_values
        dados.append({
            "data": dims[0].value if len(dims) > 0 else "N/A",
            "campanha": dims[1].value if len(dims) > 1 else "N/A",
            "fonte": dims[2].value if len(dims) > 2 else "N/A",
            "meio": dims[3].value if len(dims) > 3 else "N/A",
            "sessoes": mets[0].value if len(mets) > 0 else "0",
            "novos_usuarios": mets[1].value if len(mets) > 1 else "0",
            "usuarios_totais": mets[2].value if len(mets) > 2 else "0",
            "sessoes_engajadas": mets[3].value if len(mets) > 3 else "0",
            "taxa_rejeicao": mets[4].value if len(mets) > 4 else "0",
            "duracao_media": mets[5].value if len(mets) > 5 else "0"
        })

    return pd.DataFrame(dados)

In [347]:
df_bloco2 = run_bloco2_engajamento(client, GA4_PROPERTY_ID, DATE_RANGE)

# Conversão para float
colunas_numericas = [
    "sessoes", "novos_usuarios", "usuarios_totais",
    "sessoes_engajadas", "taxa_rejeicao", "duracao_media"
]
df_bloco2[colunas_numericas] = df_bloco2[colunas_numericas].apply(pd.to_numeric, errors="coerce")

# Exportação formatada
df_bloco2.to_csv("bloco2_engajamento.csv", index=False, sep=";", encoding="utf-8", decimal=",")

<h3>BLOCO 3: EVENTOS_COMPORTAMENTO

In [361]:
def run_bloco3_eventos_comportamento(client, property_id, date_range):
    request = RunReportRequest(
        property=f"properties/{property_id}",
        dimensions=[
            Dimension(name="date"),
            Dimension(name="eventName"),
            Dimension(name="sessionSource"),
            Dimension(name="sessionMedium")
        ],
        metrics=[
            Metric(name="eventCount"),
            Metric(name="activeUsers"),
            Metric(name="engagedSessions"),
            Metric(name="transactions")
        ],
        date_ranges=[DateRange(start_date=date_range["start"], end_date=date_range["end"])]
    )

    rows = client.run_report(request).rows
    dados = []

    for row in rows:
        dims = row.dimension_values
        mets = row.metric_values
        dados.append({
            "data": dims[0].value if len(dims) > 0 else "N/A",
            "evento": dims[1].value if len(dims) > 1 else "N/A",
            "fonte": dims[2].value if len(dims) > 2 else "N/A",
            "meio": dims[3].value if len(dims) > 3 else "N/A",
            "eventos": mets[0].value if len(mets) > 0 else "0",
            "usuarios_ativos": mets[1].value if len(mets) > 1 else "0",
            "sessoes_engajadas": mets[2].value if len(mets) > 2 else "0",
            "transacoes": mets[3].value if len(mets) > 3 else "0"
        })

    return pd.DataFrame(dados)

In [365]:
df_bloco3 = run_bloco3_eventos_comportamento(client, GA4_PROPERTY_ID, DATE_RANGE)

# Converter numéricos
colunas_numericas = ["eventos", "usuarios_ativos", "sessoes_engajadas", "transacoes"]
df_bloco3[colunas_numericas] = df_bloco3[colunas_numericas].apply(pd.to_numeric, errors="coerce")

# Exportar para CSV pronto pro Power BI
df_bloco3.to_csv("bloco3_eventos.csv", index=False, sep=";", encoding="utf-8", decimal=",")

<h3>BLOCO 4: TECNOLOGIA_GEOLOCALIZACAO

In [367]:
def run_bloco4_tecnologia_geolocalizacao(client, property_id, date_range):
    request = RunReportRequest(
        property=f"properties/{property_id}",
        dimensions=[
            Dimension(name="date"),
            Dimension(name="deviceCategory"),
            Dimension(name="browser"),
            Dimension(name="country"),
            Dimension(name="region")
        ],
        metrics=[
            Metric(name="sessions"),
            Metric(name="engagedSessions"),
            Metric(name="bounceRate"),
            Metric(name="averageSessionDuration"),
            Metric(name="transactions"),
            Metric(name="totalRevenue")
        ],
        date_ranges=[DateRange(start_date=date_range["start"], end_date=date_range["end"])]
    )

    rows = client.run_report(request).rows
    dados = []

    for row in rows:
        dims = row.dimension_values
        mets = row.metric_values
        dados.append({
            "data": dims[0].value if len(dims) > 0 else "N/A",
            "dispositivo": dims[1].value if len(dims) > 1 else "N/A",
            "navegador": dims[2].value if len(dims) > 2 else "N/A",
            "pais": dims[3].value if len(dims) > 3 else "N/A",
            "regiao": dims[4].value if len(dims) > 4 else "N/A",
            "sessoes": mets[0].value if len(mets) > 0 else "0",
            "sessoes_engajadas": mets[1].value if len(mets) > 1 else "0",
            "taxa_rejeicao": mets[2].value if len(mets) > 2 else "0",
            "duracao_media": mets[3].value if len(mets) > 3 else "0",
            "transacoes": mets[4].value if len(mets) > 4 else "0",
            "receita": mets[5].value if len(mets) > 5 else "0"
        })

    return pd.DataFrame(dados)

In [369]:
df_bloco4 = run_bloco4_tecnologia_geolocalizacao(client, GA4_PROPERTY_ID, DATE_RANGE)

# Conversão numérica
colunas_numericas = ["sessoes", "sessoes_engajadas", "taxa_rejeicao", "duracao_media", "transacoes", "receita"]
df_bloco4[colunas_numericas] = df_bloco4[colunas_numericas].apply(pd.to_numeric, errors="coerce")

# Exportar para CSV
df_bloco4.to_csv("bloco4_tecnologia.csv", index=False, sep=";", encoding="utf-8", decimal=",")

<h3>BLOCO 5: PRODUTOS_ECOMMERCE

In [379]:
def run_bloco5_produtos_ecommerce(client, property_id, date_range):
    request = RunReportRequest(
        property=f"properties/{property_id}",
        dimensions=[
            Dimension(name="date"),
            Dimension(name="itemName"),
            Dimension(name="itemCategory"),
            Dimension(name="sessionSource"),
            Dimension(name="sessionMedium")
        ],
        metrics=[
            Metric(name="itemRevenue"),
            Metric(name="itemPurchaseQuantity"),
            Metric(name="cartToViewRate"),
            Metric(name="purchaseToViewRate")
        ],
        date_ranges=[DateRange(start_date=date_range["start"], end_date=date_range["end"])]
    )

    rows = client.run_report(request).rows
    dados = []

    for row in rows:
        dims = row.dimension_values
        mets = row.metric_values
        dados.append({
            "data": dims[0].value if len(dims) > 0 else "N/A",
            "produto": dims[1].value if len(dims) > 1 else "N/A",
            "categoria": dims[2].value if len(dims) > 2 else "N/A",
            "fonte": dims[3].value if len(dims) > 3 else "N/A",
            "meio": dims[4].value if len(dims) > 4 else "N/A",
            "receita_item": mets[0].value if len(mets) > 0 else "0",
            "quantidade_vendida": mets[1].value if len(mets) > 1 else "0",
            "taxa_carrinho": mets[2].value if len(mets) > 2 else "0",
            "taxa_compra": mets[3].value if len(mets) > 3 else "0"
        })

    return pd.DataFrame(dados)

In [383]:
df_bloco5 = run_bloco5_produtos_ecommerce(client, GA4_PROPERTY_ID, DATE_RANGE)

colunas_numericas = ["receita_item", "quantidade_vendida", "taxa_carrinho", "taxa_compra"]
df_bloco5[colunas_numericas] = df_bloco5[colunas_numericas].apply(pd.to_numeric, errors="coerce")

df_bloco5.to_csv("bloco5_produtos.csv", index=False, sep=";", encoding="utf-8", decimal=",")
